**Script para separar los transcritos non coding del ensamblado**

**Javier Gutiérrez Corral**
**09/12/2023**

In [6]:
# Se importan los paquetes necesarios

import pandas as pd

In [7]:
# Se abre el documento que se ha generado al separar por coding y non coding

with open("hnoncoding.txt", 'r') as f:
    
    #Se crea el dataframe necesario
    columns=["ID"]
    tableid2=pd.DataFrame(columns=columns)
    
    #Se crea una lista en la que se guardas cada linea del documento
    file=f.readlines()
    
    #lista vacia para ir guardando los identificadores de cada transcrito
    ID=[]
    
    #Se itera sobre la lista, el identificador está separado del resto de información por tabuladores, de esta forma
    #se pueden guardar
    
    #Se guarda solo el identificador
    for i in file:
        fin=i.find("\t")
        ID.append(i[0:fin])

In [8]:
# Se guarda en el dataframe los ID

tableid2["ID"]=ID

In [9]:
'''
Se abre el fasta con todos los transcritos generados en el assembly, el objetivo es guardar en cada entrada del 
diccionario el identificador y la secuencia correspondiente.
'''

with open("hard_filtered_transcripts.fasta", 'r') as f:
    dic={}
    file=f.readlines()
    count=0
    
    #Al ser un fasta el identificador viene siempre precedido por ">"
    for i in file:
        if i[0]==">":
            fin=i.find("\n")
            name=i[0:fin]
            dic[name]=[]
        else:
            dic[name].append(i.replace('\n',''))

In [10]:
#Se ha guardado en el diccionario las secuencias en forma de lista, de esta forma las juntamos todas en una unica string

for i in dic:
    dic[i]="".join(dic[i])

In [11]:
'''
Ahora se tiene que guardar en una lista los transcritos que sean noncoding, al iterar sobre los identificadores 
del dataframe que pertenecen a secuencias noncoding,se puede buscar dicho identificador en el diccionario y guardar esa 
secuencia en una lista que posteriormente se guardará en el dataframe
'''

noncoding=[]
for i in tableid2["ID"]:
    name=">"+i
    noncoding.append(dic[name])

In [12]:
tableid2["Sequence"]=noncoding

In [13]:
tableid2.head()

,ID,Sequence
0,NODE_56_length_16476_cov_24.731756_g17_i1,AGATCTGGGAAGCGGCAGCCACATCGGGGACTGGTGGTCCGCTCCC...
1,NODE_60_length_16326_cov_476.574971_g20_i0,AGCAGAGAATTATATGTGCTTTTGTAAATAAATTAAAAGATCTTGT...
2,NODE_66_length_16238_cov_471.756573_g20_i4,AGCAGAGAATTATATGTGCTTTTGTAAATAAATTAAAAGATCTTGT...
3,NODE_82_length_14978_cov_171.135793_g32_i0,CGACCACCGAAAAGAATGTTAGTGTAGAAGCAAAGATAGTAGACAG...
4,NODE_87_length_14644_cov_249.505044_g35_i0,GCCAGTTCTGTAAGGTGACTGTACTACCATATGGTCAAATGATTAA...


In [14]:
#De esta forma se guarda en el formato fasta nuestras secuencias noncoding.
with open("HNonCodingGenes.fasta", 'w') as f:
    for i,j in zip(tableid2["Sequence"],tableid2["ID"]):
        name=">"+j
        f.write(name)
        f.write("\n")
        f.write(i)
        f.write("\n")

In [15]:
#Comprobar que el número de transcritos sea igual que en el paso de CPC2 (usado para separar anteriormente)

tableid2.count(axis=1)

0         2
1         2
2         2
3         2
4         2
         ..
616147    2
616148    2
616149    2
616150    2
616151    2
Length: 616152, dtype: int64